In [1]:
import tkinter as tk
from tkinter import filedialog

# Создаем скрытое главное окно
root = tk.Tk()
root.withdraw()

# Открываем диалог выбора файла
file_path = filedialog.askopenfilename(
    initialdir='./lib',
    title="Выберите Excel файл",
    filetypes=[("FB2 files", "*.fb2"), ("All files", "*.*")]
)

if file_path:
    # df = pd.read_csv(file_path, encoding='utf-8')
    # DataFrame df теперь содержит данные из файла
    print(f"Выбран файл: {file_path}")
else:
    print("Файл не выбран.")

# Закрываем главное окно
root.destroy()

Выбран файл: D:/Workspace/homelib/lib/d.fb2-009373-367300.zip


In [2]:
import os
import zipfile
from lxml import etree
import pandas as pd

def collect_ns_info_zip(file_path):
    records = []
    with zipfile.ZipFile(file_path) as archive:
        for fname in archive.namelist():
            if fname.endswith(".fb2") and not fname.endswith("/"):
                try:
                    with archive.open(fname) as fileobj:
                        for event, elem in etree.iterparse(fileobj, events=("start",)):
                            nsmap = dict(elem.nsmap)
                            root_tag = elem.tag
                            info = {
                                "zipfile": os.path.basename(file_path),
                                "xml_filename": fname,
                                "root_tag": root_tag
                            }
                            for k, v in nsmap.items():
                                info[f'ns_{k if k else "default"}'] = v
                            records.append(info)
                            break
                except Exception as e:
                    records.append({
                        "zipfile": os.path.basename(file_path),
                        "xml_filename": fname,
                        "root_tag": None,
                        "error": str(e)
                    })
    return pd.DataFrame(records)

def collect_all_ns_zip(file_name):
    folder = os.path.dirname(file_name)
    # Список zip-файлов в папке
    zip_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".zip") and os.path.isfile(os.path.join(folder, f))]
    all_result = pd.DataFrame()
    for zip_path in zip_files:
        df = collect_ns_info_zip(zip_path)
        all_result = pd.concat([all_result, df], ignore_index=True)
    return all_result

# Пример вызова:
df = collect_all_ns_zip(file_path)


In [ ]:
import os
from lxml import etree
import pandas as pd

def collect_ns_info(file_path):
    folder = os.path.dirname(file_path)
    files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    records = []
    for fname in files:
        try:
            # Только стартовый элемент (корень)
            for event, elem in etree.iterparse(fname, events=("start",)):
                nsmap = dict(elem.nsmap)
                root_tag = elem.tag
                info = {
                    'filename': fname,      # Полное имя файла
                    'root_tag': root_tag
                }
                # Добавляем namespace, включая default
                for k, v in nsmap.items():
                    info[f'ns_{k if k else "default"}'] = v
                records.append(info)
                break  # Корневого элемента достаточно!
        except Exception as e:
            records.append({'filename': fname, 'root_tag': None, 'error': str(e)})
    df = pd.DataFrame(records)
    return df

# Использование:
df = collect_ns_info(file_path)

In [ ]:
import zipfile
from lxml import etree
import pandas as pd

def collect_ns_info_zip(zip_path):
    records = []
    with zipfile.ZipFile(zip_path) as zf:
        # Получаем имена файлов внутри архива (только файлы)
        for fname in zf.namelist():
            # Игнорируем директории — только файлы, например, .xml
            if fname.endswith('/') or not fname.endswith('.fb2'):
                continue
            try:
                # Читаем файл из архива как байты
                with zf.open(fname) as fileobj:
                    # Парсим только первый (корневой) элемент
                    for event, elem in etree.iterparse(fileobj, events=("start",)):
                        nsmap = dict(elem.nsmap)
                        root_tag = elem.tag
                        info = {
                            'filename': fname,
                            'root_tag': root_tag
                        }
                        for k, v in nsmap.items():
                            info[f'ns_{k if k else "default"}'] = v
                        records.append(info)
                        break
            except Exception as e:
                records.append({'filename': fname, 'root_tag': None, 'error': str(e)})
    df = pd.DataFrame(records)
    return df

df = collect_ns_info_zip(file_path)

In [ ]:
from lxml import etree

context = etree.iterparse(file_path, events=("end",), tag="{*}description")
for event, elem in context:
    print(f"Получен тег: {elem.tag}, текст: {elem.text}")
    # elem — тот элемент, тэг которого только что закрылся
    elem.clear()  # освобождает память для уже обработанных элементов


In [ ]:
from lxml import etree

def extract_description_subtree(path):
    # Если в файле есть namespace по умолчанию, можно искать по локальному имени
    tag_filter = "{*}description"  # поддерживается в lxml как wildcard
    # tag_filter = tag='{http://www.gribuser.ru/xml/fictionbook/2.0}description'       # если namespaces нет

    # Событие 'end' гарантирует, что элемент полностью разобран
    context = etree.iterparse(path, events=('end',), tag=tag_filter)

    for event, elem in context:       
        return elem
    return None  # если description не найден

desc = extract_description_subtree(file_path)


In [ ]:
def element_to_flat_dict(elem):
    data = {}

    def add_value(path, val):
        if val is None:
            return
        val = val.strip()
        if not val:
            return
        if path in data:
            if isinstance(data[path], list):
                data[path].append(val)
            else:
                data[path] = [data[path], val]
        else:
            data[path] = val

    def walk(node, path):
        # локальное имя тега без namespace
        tag = etree.QName(node).localname
        cur = f"{path}/{tag}" if path else tag

        # атрибуты
        for k, v in node.attrib.items():
            add_value(f"{cur}[@{k}]", v)

        # текст узла
        if node.text and node.text.strip():
            add_value(cur, node.text)

        # дети
        for child in node:
            walk(child, cur)

        # хвостовой текст (редко нужно для FB2, но пусть будет)
        if node.tail and node.tail.strip():
            add_value(f"{path}#tail" if path else "#tail", node.tail)

    walk(elem, "")
    return data

flat_dict = element_to_flat_dict(desc)